### Speeches

Adapted from: Judd, Nicholas, Dan Drinkard, Jeremy Carbaugh, and Lindsay Young. *congressional-record: A parser for the Congressional Record.* Chicago, IL: 2017, https://github.com/unitedstates/congressional-record.

Please see LICENCE.

In [191]:
import glob
root_dir = '/home/ubuntu/Notebooks/Data_speeches'
filename_list = []
for filename in glob.glob(root_dir + '/2016/**/json/*.json', recursive=True):
    filename_list.append(filename)

In [192]:
# Get speeches from files
import os
import json
from pandas.io.json import json_normalize

speeches, speeches_json = [], []
for filename in filename_list:
    with open(filename) as f:
        speech = json.load(f)
        f.close()
        content = json_normalize(speech, 'content', meta = ['id','doc_title', 'title'])
        header = json_normalize(speech['header'])
        header['id'] = content['id']
        speech_normalized = content.merge(header, on = 'id')
        speeches.append(speech_normalized)

In [193]:
import pandas as pd
speeches_df = pd.concat(speeches, axis = 0, ignore_index = True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [234]:
# Select speeches meant to persuade (instead of procedural speeches)
print("Number of speeches of all document titles: ", speeches_df.shape[0])
speeches_to_drop =  (['TEXT OF AMENDMENTS', 'NOMINATIONS', 'ADDITIONAL COSPONSORS', 'CONFIRMATIONS', 
                      'PERSONAL EXPLANATION', 'SENATE COMMITTEE MEETINGS', 
                      'INTRODUCTION OF BILLS AND JOINT RESOLUTIONS', 'AUTHORITY FOR COMMITTEES TO MEET', 'PLEDGE OF ALLEGIANCE'])
substrings_to_drop = (['RECOGNIZING', 'REMEMBERING', 'THANKING', 'Constitutional Authority Statement', 
                       'ADJOURNMENT', 'TRIBUTE', 'HONORING', 'CONGRATULATING'])
for substring in substrings_to_drop:
    speeches_df = speeches_df[~speeches_df['doc_title'].str.contains(substring)]

speeches_df2 = speeches_df[~speeches_df['doc_title'].isin(speeches_to_drop)]
print("Number of speeches of all document titles: ", speeches_df2.shape[0])

Number of speeches of all document titles:  136148
Number of speeches of all document titles:  110269


In [238]:
# Select speeches meant to persuade (instead of procedural speeches)
print("Number of speeches of all kinds: ", speeches_df2.shape[0])
print(speeches_df2['kind'].value_counts())
#list(speeches_df[(speeches_df['kind'] == 'Unknown')]['text'])
speeches_df3 = speeches_df2[(speeches_df2['kind'] == 'speech') | (speeches_df2['kind'] == 'title')]
print("Number of speeches (speech): ", speeches_df3.shape[0])
speeches_df3.reset_index(inplace = True)

Number of speeches of all kinds:  110269
speech            49661
title             26012
linebreak         16751
recorder          11589
Unknown            4798
clerk               823
metacharacters      632
empty_line            3
Name: kind, dtype: int64
Number of speeches (speech):  75673


In [196]:
speeches_df3.head()

,index,chamber,day,doc_title,extension,id,itemno,kind,month,num,pages,speaker,speaker_bioguide,text,title,turn,vol,wkday,year
0,5135,Senate,14,EXECUTIVE SESSION,False,CREC-2016-07-14-pt1-PgS5181-4,4,speech,July,114,S5181-S5182,Ms. MURKOWSKI,M001153,"Ms. MURKOWSKI. Mr. President, I ask unanimou...",EXECUTIVE SESSION,0,162,Thursday,2016
1,5136,Senate,14,EXECUTIVE SESSION,False,CREC-2016-07-14-pt1-PgS5181-4,5,speech,July,114,S5181-S5182,The PRESIDING OFFICER,None,"The PRESIDING OFFICER. Without objection, it...",EXECUTIVE SESSION,1,162,Thursday,2016
2,5138,Senate,14,EXECUTIVE SESSION,False,CREC-2016-07-14-pt1-PgS5181-4,7,speech,July,114,S5181-S5182,Ms. MURKOWSKI,M001153,"Ms. MURKOWSKI. Mr. President, I ask for a di...",EXECUTIVE SESSION,2,162,Thursday,2016
3,5139,Senate,14,EXECUTIVE SESSION,False,CREC-2016-07-14-pt1-PgS5181-4,8,speech,July,114,S5181-S5182,The PRESIDING OFFICER,None,The PRESIDING OFFICER. A division vote has b...,EXECUTIVE SESSION,3,162,Thursday,2016
4,5354,Senate,14,LEGISLATIVE SESSION,False,CREC-2016-07-14-pt1-PgS5182-2,0,speech,July,114,S5182,The PRESIDING OFFICER,None,The PRESIDING OFFICER. The Senate will now r...,LEGISLATIVE SESSION,0,162,Thursday,2016


In [197]:
# Cleaning the speech text
speeches_df3['text'] = [text.replace('\n', '').replace('  ', '') for text in speeches_df3['text']]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [198]:
import datetime
import pandas as pd
speeches_df3['month'] = [datetime.datetime.strptime(month, '%B').strftime('%m') for month in speeches_df3['month']]
speeches_df3['date'] = pd.to_datetime(speeches_df3[['year','month','day']])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [199]:
import pandas as pd
speeches_df4 = speeches_df3.loc[:,~speeches_df3.columns.duplicated()]

"\n# Extracting time / date features\n# month / year\nspeeches_df4 = speeches_df4.assign(year_month=pd.to_datetime(speeches_df4[['year', 'month']].assign(day=1)))\n# how long ago?\ndate_now = pd.to_datetime('2018-07-27')\nspeeches_df4['days_ago'] = [(date_now - date).days for date in speeches_df4['date']]\n\nspeeches_df4 = speeches_df4[['id','speaker_bioguide', 'text', 'date', 'wkday', 'year_month', 'days_ago']] \n"

In [200]:
speeches_df4 = speeches_df3[['chamber', 'doc_title', 'extension', 'id', 'itemno',
        'num', 'pages', 'speaker', 'speaker_bioguide', 'text',
       'title', 'turn', 'date']]

In [201]:
speeches_df4.drop_duplicates(inplace = True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [202]:
speeches_df4.shape

(635, 13)

In [203]:
import pickle
output = open('speeches_cleaned_2016.pkl', 'wb')
pickle.dump(speeches_df4, output)

output.close()

In [34]:
import pickle
output = open('speeches_cleaned.pkl', 'wb')
pickle.dump(speeches_df4, output)

output.close()

### Merge in metadata

In [253]:
# Already created metadata for 2017-8
# Create list of files of metadata to parse
import glob
root_dir = '/home/ubuntu/Notebooks/Data_speeches'
metadata_list = []
for metadata in glob.glob(root_dir + '/2014/**/mods.xml', recursive=True):
    metadata_list.append(metadata)
for metadata in glob.glob(root_dir + '/2015/**/mods.xml', recursive=True):
    metadata_list.append(metadata)
for metadata in glob.glob(root_dir + '/2016/**/mods.xml', recursive=True):
    metadata_list.append(metadata)

In [254]:
len(metadata_list)

527

In [255]:
# Explode a list inside a Dataframe cell into separate rows
def stack_unstack(df, col_target):
    return (df[col_target].apply(pd.Series)
            .stack()
            .reset_index(level=1, drop=True)
            .to_frame(col_target))

In [256]:
# Convert xml into a pandas dataframe
from bs4 import BeautifulSoup
from collections import defaultdict
import pandas as pd

for idx, meta_data in enumerate(metadata_list):
    speeches_bill_ids_all = pd.DataFrame()
    if idx % 50 == 0:
        print("Progress: ", idx / len(metadata_list) * 100, "%")
    with open(meta_data, 'r') as f:
        mods_data = f.read()
    f.close()
    soup = BeautifulSoup(mods_data, 'lxml')
    list_dicts = []
    bill_accessid_dict = defaultdict(list)
    for i in soup.find_all('bill'):
        meta_data_dict = {}
        for data in ['congress', 'number', 'type']:
            meta_data_dict[data] = i[data]
        #print(i)
        bill = [b.parent for b in soup.find_all('bill', congress = i['congress'], context = i['context'], number = i['number'], type = i['type'])]
        #print(bill)
        try:
            accessid = [b.find('accessid') for b in bill]
            accessid_list = [a.text for a in accessid]
            meta_data_dict['accessids'] = accessid_list
        except:
            continue
        list_dicts.append(meta_data_dict)
        meta_data_df = pd.DataFrame(list_dicts)
        accessids_unstacked = stack_unstack(meta_data_df, 'accessids')
        speeches_bill_ids = meta_data_df[['congress', 'number', 'type']].merge(accessids_unstacked, left_index = True, right_index = True)
        if speeches_bill_ids_all.empty is True:
            speeches_bill_ids_all = speeches_bill_ids
        else:
            speeches_bill_ids_all = pd.concat([speeches_bill_ids_all,speeches_bill_ids], axis = 0, join = 'outer', ignore_index = True)
    try:
        speeches_bill_ids_all.drop_duplicates(subset = ['congress', 'number', 'type'], inplace = True)
    except(KeyError):
        continue
    speeches_bill_ids_all.to_csv('metadata/metadata_{idx}'.format(idx = idx))

Progress:  0.0 %
Progress:  9.487666034155598 %
Progress:  18.975332068311197 %
Progress:  28.46299810246679 %
Progress:  37.95066413662239 %
Progress:  47.43833017077799 %
Progress:  56.92599620493358 %
Progress:  66.41366223908919 %
Progress:  75.90132827324479 %
Progress:  85.38899430740038 %
Progress:  94.87666034155598 %


In [289]:
pkl_file = open('speeches_cleaned.pkl', 'rb')
speeches_cleaned = pickle.load(pkl_file)

unique_ids = list(speeches_cleaned['id'].unique())

In [301]:
speeches_cleaned.shape

(409395, 13)

In [259]:
# Concatenate all of the parsed metadata
import glob
import pandas as pd
root_dir = '/home/ubuntu/Notebooks/metadata'
metadata_df_all = pd.DataFrame()
for metadata in glob.glob(root_dir + '/*'):
    metadata_df = pd.read_csv(metadata)
    # Drop the metadata if it doesn't correspond to an id for the selected speeches
    metadata_df = metadata_df[metadata_df['accessids'].isin(unique_ids)]
    if metadata_df_all.empty is True:
        metadata_df_all = metadata_df
    else:
        metadata_df_all = pd.concat([metadata_df_all, metadata_df], axis = 0, ignore_index = True)
        metadata_df_all = metadata_df_all.drop_duplicates(subset = ['accessids', 'congress', 'number', 'type'])

In [260]:
# Format bill ids to match the bill ids taken in by the API
metadata_df_all['type'] = [t.lower() for t in metadata_df_all['type']]
metadata_df_all['bill_id'] = metadata_df_all['type'] + metadata_df_all['number'].map(int).map(str)

metadata_df_all = metadata_df_all[['accessids', 'bill_id']]

In [318]:
speeches_cleaned['index'] = speeches_cleaned.index

In [320]:
# Merge speeches and metadata
speeches_metadata = speeches_cleaned.merge(metadata_df_all, left_on = 'id', right_on = 'accessids', how = 'inner')

In [293]:
#speeches_metadata.groupby('bill_id')['text'].count()

In [322]:
import pickle
output = open('speeches_metadata.pkl', 'wb')
pickle.dump(speeches_metadata, output)
output.close()

### Extracting features from the speeches

#### Mentions think tank

In [ ]:
# Source: http://www.citizensource.com/Opinion&Policy/ThinkTanks.htm
from bs4 import BeautifulSoup

think_tanks_response = requests.get('http://www.citizensource.com/Opinion&Policy/ThinkTanks.htm')
think_tanks_text = think_tanks_response.text
think_tanks_soup = BeautifulSoup(think_tanks_text, "lxml")

In [ ]:
tables = []
for table in think_tanks_soup.find_all('a'): 
    think_tank = " ".join(table.text.split())
    #print(think_tank)
    tables.append(think_tank)

In [ ]:
import itertools
def isplit(iterable,splitters):
    return [list(g) for k,g in itertools.groupby(iterable,lambda x:x in splitters) if not k]

think_tank_list = isplit(tables,'p')

think_tank_orientation = think_tank_list[0]
think_tank_dict = {}
for idx, think_tank in enumerate(think_tank_list[1:]):
    think_tank_dict[think_tank_orientation[idx]] = think_tank

In [ ]:
from collections import defaultdict
republican = ['RIGHT', 'LIBERTARIAN', 'CONSERVATIVE']
democrat = ['LIBERAL', 'LEFT']
think_tank_party_dict = defaultdict(list)
for key, value in think_tank_dict.items(): 
    if key in republican:
        think_tank_party_dict['Republican'].append(value)
    elif key in democrat:
        think_tank_party_dict['Democrat'].append(value)
    else:
        think_tank_party_dict['Non-partisan'].append(value)

In [ ]:
for key, value in think_tank_party_dict.items():
    think_tank_party_dict[key] = [item for sublist in value for item in sublist]

In [ ]:
import pickle
output = open('think_tanks.pkl', 'wb')
pickle.dump(think_tank_party_dict, output)

output.close()

In [324]:
pkl_file = open('think_tanks.pkl', 'rb')
think_tank_party_dict = pickle.load(pkl_file)

pkl_file = open('speeches_metadata.pkl', 'rb')
speeches_cleaned = pickle.load(pkl_file)

In [325]:
print(speeches_cleaned.shape)
print(len(speeches_cleaned['text'].unique()))
unique_speeches = speeches_cleaned['text'].unique()

(976770, 16)
131203


In [326]:
speeches_cleaned.reset_index(inplace = True)

In [330]:
# Fuzzy matching of mentioning think tank name
from fuzzywuzzy import process
speeches_cleaned_tt = (pd.DataFrame(columns = ['text','mentions_think_tank_Republican', 
                        'mentions_think_tank_Democrat', 'mentions_think_tank_Non-partisan',
                        'mentions_think_tank_Republican_no', 'mentions_think_tank_Democrat_no',
                          'mentions_think_tank_Non-partisan_no']))

for idx, sp in enumerate(unique_speeches):
    for key, value in think_tank_party_dict.items():
        speeches_cleaned_tt.at[idx,'text'] = sp
        fuzzy_name = [process.extractOne(sp, v) for v in value]
        fuzzies = [fuzzies[0] if fuzzies[1] >=95 else '' for fuzzies in fuzzy_name]
        if all(v for v in fuzzies) is True:
            print(sp)
            print(fuzzies)
            speeches_cleaned_tt.at[idx,'mentions_think_tank_{party}'.format(party = key)] = fuzzies
            speeches_cleaned_tt.at[idx,'mentions_think_tank_{party}_no'.format(party = key)] = len(fuzzies)       

In [331]:
speeches_cleaned = speeches_cleaned.merge(speeches_cleaned_tt, on = 'text')

#### Mentions of non-partisan agencies which provide research

In [332]:
import re
non_partisan_agency = re.compile('''CRS|C\s+R\s+S|C.R.S|C.\s+R.\s+S.|Congressional Research Service|
                                CBO|C\s+B\s+O|C.B.O|C.\s+B.\s+O.|Congressional Budget Office|
                                GAO|G\s+A\s+O|G.A.O|G.\s+A.\s+O.|Government Accountability Office''')
speeches_cleaned['mentions_non_partisan_agency'] = ''
speeches_cleaned['mentions_non_partisan_agency_no'] = 0
for idx, sp in enumerate(speeches_cleaned['text']):
    agencies_mentioned = re.findall(non_partisan_agency,speeches_cleaned.loc[idx,'text'])
    speeches_cleaned.at[idx,'mentions_non_partisan_agency'] = agencies_mentioned
    speeches_cleaned.at[idx,'mentions_non_partisan_agency_no'] = len(agencies_mentioned)

#### Mentions statistics

In [333]:
import re
# Only keep relevant numbers which relate to statistics
numbers_keep = re.compile('''[0-9]+(?=\s+percent)|one-quarter|one-third|one-half|two-quarters|two-thirds''')
# Find all numbers to check I'm not missing any relevant patterns
numbers_all = re.compile('[a-zA-Z]+[\s\!\"\#\$\%\&\\\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~]+[0-9]+[\s\!\"\#\$\%\&\\\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~]+[a-zA-Z]+')
speeches_cleaned['uses_statistics'] = 0
for idx, sp in enumerate(speeches_cleaned['text']):
    speeches_cleaned.at[idx,'uses_statistics'] = len(re.findall(numbers_keep,speeches_cleaned.loc[idx,'text']))

In [334]:
speeches_cleaned.columns

Index(['level_0', 'chamber', 'doc_title', 'extension', 'id', 'itemno', 'num',
       'pages', 'speaker', 'speaker_bioguide', 'text', 'title', 'turn', 'date',
       'index', 'accessids', 'bill_id', 'mentions_think_tank_Republican',
       'mentions_think_tank_Democrat', 'mentions_think_tank_Non-partisan',
       'mentions_think_tank_Republican_no', 'mentions_think_tank_Democrat_no',
       'mentions_think_tank_Non-partisan_no', 'mentions_non_partisan_agency',
       'mentions_non_partisan_agency_no', 'uses_statistics'],
      dtype='object')

In [337]:
max(speeches_cleaned['index'])

106255

In [274]:
# Dropping text - no need for the text 
speeches_cleaned = (speeches_cleaned[['mentions_think_tank_Republican_no', 'mentions_think_tank_Democrat_no',
                                    'mentions_think_tank_Non-partisan_no', 'mentions_non_partisan_agency_no',
                                    'uses_statistics', 'id','speaker_bioguide', 'date', 'index']])

In [338]:
import pickle
output = open('speeches_features.pkl', 'wb')
pickle.dump(speeches_cleaned, output)

output.close()

### Merging speeches and metadata

- Calculate summary stats for speeches

In [29]:
import pickle
import pandas as pd

pkl_file = open('speeches_features.pkl', 'rb')
speeches_features = pickle.load(pkl_file)

pkl_file = open('speeches_similarity.pkl', 'rb')
speeches_similarity = pickle.load(pkl_file)

In [38]:
speeches_similarity_df = pd.DataFrame(speeches_similarity)
speeches_similarity_df['index'] = speeches_similarity_df.index

In [120]:
speeches_metadata = speeches_features.merge(speeches_similarity_df, on = 'index')

In [94]:
speeches_metadata.columns

Index(['level_0', 'chamber', 'doc_title', 'extension', 'id', 'itemno', 'num',
       'pages', 'speaker', 'speaker_bioguide', 'text', 'title', 'turn', 'date',
       'index', 'accessids', 'bill_id', 'mentions_think_tank_Republican',
       'mentions_think_tank_Democrat', 'mentions_think_tank_Non-partisan',
       'mentions_think_tank_Republican_no', 'mentions_think_tank_Democrat_no',
       'mentions_think_tank_Non-partisan_no', 'mentions_non_partisan_agency',
       'mentions_non_partisan_agency_no', 'uses_statistics', 'crs_sim_avg'],
      dtype='object')

In [136]:
# Congress
import datetime
start_end_113 = (datetime.date(2013, 1, 3), datetime.date(2015, 1, 3))
start_end_114 = (datetime.date(2015, 1, 6), datetime.date(2017, 1, 3))
start_end_115 = (datetime.date(2017, 1, 3), datetime.date(2019, 1, 3))
start_end_list = [start_end_113, start_end_114, start_end_115]
congress_no = list(range(113,116))    
start_end_dict = dict(zip(congress_no, start_end_list))
speeches_metadata['date'] = [x.date()if type(x) is not datetime.date else x for x in speeches_metadata['date'] ]
for congress, start_end in start_end_dict.items():
    mask = (speeches_metadata['date'] > start_end[0]) & (speeches_metadata['date'] <= start_end[1])
    speeches_metadata.loc[mask,'congress'] = congress

In [140]:
speeches_metadata.loc[speeches_metadata['congress'].isnull().sum(), 'congress'] = 114

In [141]:
# Include these summary statistics

evidence_keys = ['mentions_non_partisan_agency_no', 'uses_statistics', 'crs_sim_avg']

evidence_stats = ['mean', 'median', 'max', 'std']

evidence_dict = {}
for key in evidence_keys:
    evidence_dict[key] = evidence_stats


In [142]:
# Calculate descriptive stats to summarise the speeches for each bill
# Create dataset whose unique id is the bill
bill_evidence_stats = speeches_metadata.groupby('bill_id').agg(evidence_dict)
speeches_metadata_evidence = speeches_metadata[['accessids', 'bill_id', 'speaker_bioguide', 'congress']].merge(bill_evidence_stats, on = 'bill_id', how = 'inner')
cols_new = []
for col in speeches_metadata_evidence.columns[4:]:
    col_new = col[0] + '_' + col[1]
    cols_new.append(col_new)
speeches_metadata_evidence.rename(columns = dict(zip(speeches_metadata_evidence.columns[4:], cols_new)), inplace = True)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:543: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [143]:
speeches_metadata_evidence.columns

Index(['accessids', 'bill_id', 'speaker_bioguide', 'congress',
       'mentions_non_partisan_agency_no_mean',
       'mentions_non_partisan_agency_no_median',
       'mentions_non_partisan_agency_no_max',
       'mentions_non_partisan_agency_no_std', 'uses_statistics_mean',
       'uses_statistics_median', 'uses_statistics_max', 'uses_statistics_std',
       'crs_sim_avg_mean', 'crs_sim_avg_median', 'crs_sim_avg_max',
       'crs_sim_avg_std'],
      dtype='object')

In [144]:
speaker_dummies = pd.get_dummies(speeches_metadata_evidence['speaker_bioguide'], prefix = 'speaker_id')
speeches_metadata_evidence = pd.concat([speeches_metadata_evidence,speaker_dummies], axis = 1)
speeches_metadata_evidence2 = speeches_metadata_evidence.drop_duplicates(subset = ['bill_id'])

In [145]:
import pickle
output = open('speeches_metadata_evidence.pkl', 'wb')
pickle.dump(speeches_metadata_evidence2, output)
output.close()

### Collect bill information if mentioned in a speech

ProPublica Congress API, https://projects.propublica.org/api-docs/congress-api/, last access: 07/30/2018.

In [168]:
import pickle
pkl_file = open('speeches_metadata_evidence.pkl', 'rb')
speeches_metadata = pickle.load(pkl_file)

In [169]:
speeches_metadata.reset_index(inplace = True)

In [170]:
speeches_metadata.shape

(3898, 693)

In [171]:
import requests
import numpy
headers = {'X-API-Key': '######'}
responses_bills = []
for idx in range(0,speeches_metadata.shape[0]):
    try:
        bill = 'https://api.propublica.org/congress/v1/{congress}/bills/{bill_id}.json'.format(congress = int(speeches_metadata.loc[idx,'congress']), bill_id = speeches_metadata.loc[idx,'bill_id'])
        response = requests.get(bill, headers = headers)
        #print(response.json())
        responses_bills.append(response.json())
    except:
        print(idx)
        continue

703
704
705
706
707
708
709
710
711
712
714
715
863
2376
3615
3616


In [172]:
# Count number of bills with votes
house_votes, senate_votes = [], []
for res in responses_bills:
    try:
        house_vote = res['results'][0]['house_passage_vote']
        if house_vote is not None:
            house_votes.append(house_vote)
        senate_vote = res['results'][0]['senate_passage_vote']
        if senate_vote is not None:
            senate_votes.append(senate_vote)
    except:
        continue
print(len(house_votes))
print(len(senate_votes))

1049
465


In [ ]:
# Create a dataframe of the bills mentioned
from pandas.io.json import json_normalize

mentioned_bills_all = pd.DataFrame()
for idx, bill in enumerate(responses_bills):
    try:
        mentioned_bills = json_normalize(bill, 'results')
        mentioned_bills_votes = json_normalize(bill, ['results', 'votes'])
        mentioned_bills_votes = mentioned_bills_votes.add_prefix('votes_')
        mentioned_bills_votes['bill_id'] = mentioned_bills['bill_id'][0]
        mentioned_bills_both = mentioned_bills.merge(mentioned_bills_votes, on = 'bill_id', how = 'outer')
        if mentioned_bills_all.empty is True:
            mentioned_bills_all = mentioned_bills_both
        else:
            mentioned_bills_all = pd.concat([mentioned_bills_all,mentioned_bills_both], axis = 0, join = 'outer', ignore_index = True)
    except(KeyError):
        print("Key error at: ", idx)
        continue

In [174]:
# Count number of bills with votes
mentioned_bills_all['votes_roll_call'].value_counts().sum()

2360

In [175]:
import pickle
output = open('mentioned_bills.pkl', 'wb')
pickle.dump(mentioned_bills_all, output)

output.close()

In [176]:
mentioned_bills_all.shape

(5541, 51)

In [177]:
mentioned_bills_all['last_vote'] = pd.to_datetime(mentioned_bills_all['last_vote'], format = '%Y-%m-%d')
mentioned_bills_all['votes_date'] = pd.to_datetime(mentioned_bills_all['votes_date'], format = '%Y-%m-%d')

In [178]:
# Create a variable to indicate that one than one party sponsored the bill
for idx, row in enumerate(mentioned_bills_all):
    if len(mentioned_bills_all.loc[idx, 'cosponsors_by_party'].keys()) > 1:
        mentioned_bills_all.loc[idx, 'cosponsored_by_mt1_party'] = 1
    else:
        mentioned_bills_all.loc[idx, 'cosponsored_by_mt1_party'] = 0

In [179]:
# Create a variable to indicate that the bill was sponsored reasonably equally (40-50% by the second largest sponsoring party)
import heapq
for idx, row in enumerate(mentioned_bills_all):
    co_sponsorship = mentioned_bills_all.loc[idx, 'cosponsors_by_party'].values()
    ratio = [v / max(co_sponsorship) for v in co_sponsorship]
    try:
        if heapq.nlargest(2, ratio)[1] > 0.4:
            mentioned_bills_all.loc[idx, 'cosponsored_ratio_mr40pc'] = 1
        else:
            mentioned_bills_all.loc[idx, 'cosponsored_ratio_mr40pc'] = 0
    except:
        mentioned_bills_all.loc[idx, 'cosponsored_ratio_mr40pc'] = 0

In [180]:
mentioned_bills_all.columns

Index(['actions', 'active', 'bill', 'bill_id', 'bill_slug', 'bill_type',
       'bill_uri', 'committee_codes', 'committees', 'congress',
       'congressdotgov_url', 'cosponsors', 'cosponsors_by_party', 'enacted',
       'govtrack_url', 'gpo_pdf_uri', 'house_passage', 'house_passage_vote',
       'introduced_date', 'last_vote', 'latest_major_action',
       'latest_major_action_date', 'number', 'primary_subject',
       'senate_passage', 'senate_passage_vote', 'short_title', 'sponsor',
       'sponsor_id', 'sponsor_party', 'sponsor_state', 'sponsor_title',
       'sponsor_uri', 'subcommittee_codes', 'summary', 'summary_short',
       'title', 'versions', 'vetoed', 'votes', 'votes_api_url',
       'votes_chamber', 'votes_date', 'votes_question', 'votes_result',
       'votes_roll_call', 'votes_time', 'votes_total_no',
       'votes_total_not_voting', 'votes_total_yes', 'withdrawn_cosponsors',
       'cosponsored_by_mt1_party', 'cosponsored_ratio_mr40pc'],
      dtype='object')

In [181]:
# Subject nicheness is a normalised frequency count of the number of times it's mentioned in Congress
# ?? Risk of differences due to small counts for nicher subjects?
mentioned_bills_all['subject_nicheness'] = mentioned_bills_all.groupby('primary_subject')['primary_subject'].transform('count') / mentioned_bills_all['primary_subject'].value_counts()[0]

# Number of months the primary subject is mentioned
# month / year
mentioned_bills_all['month'] = [x.month for x in mentioned_bills_all['votes_date']]
mentioned_bills_all['year'] = [x.year for x in mentioned_bills_all['votes_date']]
mentioned_bills_all = mentioned_bills_all.assign(year_month=pd.to_datetime(mentioned_bills_all[['year', 'month']].assign(day=1)))
df_primary_subject_trends = mentioned_bills_all.groupby(['primary_subject','year_month'])['bill_id'].count().reset_index()
df_primary_subject_trends.rename(columns = {'id': 'primary_subject_number_of_mentions'}, inplace = True)
df_primary_subject_trends_month = df_primary_subject_trends.groupby('primary_subject')['year_month'].count().reset_index()
df_primary_subject_trends_month.rename(columns = {'year_month': 'primary_suject_no_months_mentioned'}, inplace = True)
df_primary_subject_trends = df_primary_subject_trends.merge(df_primary_subject_trends_month, on = 'primary_subject')
mentioned_bills_all = mentioned_bills_all.merge(df_primary_subject_trends, on = ['primary_subject', 'year_month'])


In [182]:
mentioned_bills_all = mentioned_bills_all[['active', 'bill_slug', 'committees', 'congress',
            'enacted', 'primary_subject', 'sponsor_id','sponsor_party', 'sponsor_state', 
            'vetoed', 'votes_api_url','votes_chamber', 'votes_date','votes_time', 
            'votes_total_no', 'votes_total_not_voting','votes_total_yes', 'cosponsored_by_mt1_party',
           'cosponsored_ratio_mr40pc', 'primary_suject_no_months_mentioned', 'subject_nicheness']]

In [183]:
mentioned_bills_all['congress'].value_counts().sort_index()

113     647
114    1000
115     713
Name: congress, dtype: int64

In [184]:
import pickle
output = open('mentioned_bills.pkl', 'wb')
pickle.dump(mentioned_bills_all, output)

output.close()

### Get voting information of bills mentioned

ProPublica Congress API, https://projects.propublica.org/api-docs/congress-api/, last access: 07/30/2018.

In [185]:
import pickle
pkl_file = open('mentioned_bills.pkl', 'rb')
mentioned_bills_all = pickle.load(pkl_file)

In [186]:
mentioned_bills_all['votes_api_url_2'] = mentioned_bills_all['votes_api_url'].fillna('')

In [187]:
unique_votes = mentioned_bills_all['votes_api_url_2'].unique()

In [188]:
len(unique_votes)

2360

In [189]:
# api_url is the api call for the votes associated with the bill
import requests
import numpy as np
import math
headers = {'X-API-Key': '######'}

responses = {}
for vote_url in unique_votes:
    if vote_url != '':
        response = requests.get(vote_url, headers = headers)
        responses[vote_url] = response.json()

In [193]:
import pandas as pd
# Get votes broken down by party
multiple_party_votes_all = pd.DataFrame()
for idx, res in responses.items():
    voting_positions = pd.DataFrame(responses[idx]['results']['votes']['vote']['positions'])
    voting_positions['bill_id'] = responses[idx]['results']['votes']['vote']['bill']['bill_id']
    voting_positions['vote_api'] = idx
    voting_positions['chamber'] = responses[idx]['results']['votes']['vote']['chamber']
    voting_positions['date'] = responses[idx]['results']['votes']['vote']['date']
    voting_positions['time'] = responses[idx]['results']['votes']['vote']['time']
    multiple_party_votes = pd.DataFrame()
    for party in ['democratic', 'republican', 'independent']:
        party_votes = pd.DataFrame([responses[idx]['results']['votes']['vote'][party]])
        party_votes = party_votes.add_prefix('{party}_'.format(party = party))
        if multiple_party_votes.empty is True:
            multiple_party_votes = party_votes
        else:
            multiple_party_votes = pd.concat([multiple_party_votes, party_votes], axis = 1, join = 'outer')
    multiple_party_votes['vote_api'] = idx
    party_votes2 = multiple_party_votes.merge(voting_positions, on = 'vote_api', how = 'right')
    if multiple_party_votes_all.empty is True:
        multiple_party_votes_all = party_votes2
    else:
        multiple_party_votes_all = pd.concat([multiple_party_votes_all, party_votes2], axis = 0, join = 'outer')   

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [194]:
multiple_party_votes_all['vote_position'].value_counts()

Yes           437362
No            315032
Not Voting     26456
Speaker         1594
Present          283
Name: vote_position, dtype: int64

In [200]:
mask = ((multiple_party_votes_all['vote_position'] == 'Yes') | 
        (multiple_party_votes_all['vote_position'] == 'No') |
        (multiple_party_votes_all['vote_position'] == 'Not Voting'))
multiple_party_votes_all = multiple_party_votes_all[mask]

In [201]:
import datetime
multiple_party_votes_all['date'] = pd.to_datetime(multiple_party_votes_all['date'])

In [202]:
import datetime
import pandas as pd

date_now = pd.to_datetime('2018-07-27')
multiple_party_votes_all['days_ago'] = [(date_now - date).days for date in multiple_party_votes_all['date']]

In [203]:
multiple_party_votes_all.reset_index(inplace = True)

In [204]:
# Vote with or against the party / abstain
multiple_party_votes_all.loc[multiple_party_votes_all['vote_position'] == 'Not Voting','vote_position'] = 'No'
party_abbr_dict = {'republican': 'R', 'democratic': 'D'}
multiple_party_votes_all['vote_with_party'] = 0
for key, value in party_abbr_dict.items():
    for idx in range(0, multiple_party_votes_all.shape[0]):
        #print(multiple_party_votes_all.loc[idx, '{party}_majority_position'.format(party = key)])
        #print(multiple_party_votes_all.loc[idx,'vote_position'])
        #print(multiple_party_votes_all.loc[idx,'party'] == '{party_abbr}'.format(party_abbr = value))
        if ((multiple_party_votes_all.loc[idx, '{party}_majority_position'.format(party = key)] == multiple_party_votes_all.loc[idx,'vote_position'])
        and (multiple_party_votes_all.loc[idx,'party'] == '{party_abbr}'.format(party_abbr = value))):
            multiple_party_votes_all.loc[idx, 'vote_with_party'] = 1

In [205]:
# Independent variable
multiple_party_votes_all['vote_with_party'].value_counts()

1    708537
0     70313
Name: vote_with_party, dtype: int64

In [206]:
# Making sure both chambers are covered
from collections import Counter
chambers = []
for i in responses:
    chamber = responses[i]['results']['votes']['vote']['chamber']
    chambers.append(chamber)
d = Counter(chambers)
d

Counter({'Senate': 717, 'House': 1643})

In [207]:
# time of day
import pandas as pd
multiple_party_votes_all['votes_hour'] = [x.hour for x in pd.to_datetime(multiple_party_votes_all['time'], format = '%H:%M:%S')]
bins = [0, 5, 12, 18, 22, 24]
labels = ['early_morning', 'morning', 'afternoon', 'evening', 'night']
multiple_party_votes_all['votes_time_of_day'] = pd.cut(multiple_party_votes_all['votes_hour'], bins = bins, labels = labels)

In [208]:
multiple_party_votes_all = multiple_party_votes_all.loc[:,~multiple_party_votes_all.columns.duplicated()]

In [209]:
multiple_party_votes_all.drop(columns = ['index', 'votes_hour', 'time', 'district', 'date'], inplace = True)

In [210]:
multiple_party_votes_all.shape

(778850, 26)

In [211]:
import pickle
output = open('votes_by_party.pkl', 'wb')
pickle.dump(multiple_party_votes_all, output)

output.close()

### Party and state of member

ProPublica Congress API, https://projects.propublica.org/api-docs/congress-api/, last access: 07/30/2018.

In [52]:
import pickle

pkl_file = open('votes_by_party.pkl', 'rb')
votes_by_party = pickle.load(pkl_file)

pkl_file = open('speeches_features.pkl', 'rb')
speeches_cleaned = pickle.load(pkl_file)

In [53]:
unique_member_ids = votes_by_party['member_id'].unique()
print(len(unique_member_ids))

684


In [54]:
unique_speaker_id = speeches_cleaned['speaker_bioguide'].unique()
print(len(unique_speaker_id))

678


In [55]:
members_info_to_acq = set(unique_member_ids).union(set(unique_speaker_id))
print(len(members_info_to_acq))

695


In [56]:
# Getting information for 
import requests
headers = {'X-API-Key': '######'}
import pandas as pd
from pandas.io.json import json_normalize

members_df = pd.DataFrame()
cols = (['chamber', 'congress', 'missed_votes_pct', 'senate_class', 
            'seniority','state_rank', 'subcommittees', 'leadership_role', 
                'state', 'votes_with_party_pct', 'party'])
for idx, speaker_id in enumerate(members_info_to_acq):
    #print(speaker_id)
    if pd.isnull(speaker_id) is False:
        try:
            speaker = 'https://api.propublica.org/congress/v1/members/{speaker}.json'.format(speaker = speaker_id)
            response = requests.get(speaker, headers = headers)
            member = json_normalize(response.json(), ['results'])
            #print(member.shape)
            member = member[['current_party', 'date_of_birth', 'first_name', 'gender', 
                             'in_office', 'last_name', 'member_id']]
            member['full_name'] = member['first_name'] + ' ' + member['last_name']
            roles = json_normalize(response.json(), ['results', 'roles'])
            #print(roles.shape)
            roles_col = [col for col in roles.columns if col in cols]
            roles = roles[roles_col]
            roles['member_id'] = member['member_id'][0]
            member_roles = roles.merge(member, on = 'member_id', how = 'left')
            member_roles = member_roles.add_prefix('members_')
            if members_df.empty is True:
                members_df = member_roles
            else:
                members_df = pd.concat([members_df, member_roles], axis = 0, join = 'outer')
        except(KeyError):
            print(speaker_id, "not found")
            continue

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



None not found


In [57]:
import requests

headers = {'X-API-Key': '######'}

def get_data(url):
    responses = []
    response = requests.get(url, headers = headers)
    #print(response.json())
    responses.append(response.json())
    try:
        num_results = response.json()['num_results']
        print(num_results)
        for req in range(0,num_results,20):
            response = requests.get(url + '{num}'.format(num = req), headers = headers)
            responses.append(response.json())
        return responses
    except(KeyError):
        return responses

In [58]:
# Getting years to next election for those currently in Congress
import pandas as pd
from pandas.io.json import json_normalize

senate = 'https://api.propublica.org/congress/v1/115/senate/members.json'
house = 'https://api.propublica.org/congress/v1/115/house/members.json'

members_df_current = pd.DataFrame()
for affiliation in [senate, house]:
    party = get_data(affiliation)
    members = json_normalize(party, ['results','members'])
    results = json_normalize(party, 'results')
    if members_df_current.empty is True:
        members_df_current = members
    else:
        members_df_current = pd.concat([members_df_current, members], axis = 0, join = 'outer', ignore_index = True)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  app.launch_new_instance()


In [59]:
# combine members_df_current and members_df
list_congress = ['113', '114', '115']
members_df_recent = members_df[(members_df['members_congress'].isin(list_congress))]
members_df_current['members_yrs_until_nxt_election'] = members_df_current['next_election'].map(int) - 2018
members_df_all = members_df_recent.merge(members_df_current, left_on = 'members_member_id', right_on = 'id', how = 'left')

In [60]:
# how old is the member?
members_df_all['members_age'] =  [(2018 - x.year) for x in pd.to_datetime(members_df_all['members_date_of_birth'])]

In [63]:
members_df_all = (members_df_all[['members_chamber', 
        'members_full_name',
       'members_gender', 'members_in_office', 
       'members_leadership_role', 'members_member_id',
       'members_missed_votes_pct', 'members_party', 'members_senate_class',
       'members_seniority', 'members_state', 'members_state_rank',
        'members_votes_with_party_pct', 
       'dw_nominate',  'gender',  'leadership_role', 'missed_votes_pct', 
       'votes_with_party_pct', 'members_yrs_until_nxt_election',
       'members_age', 'members_congress']])

In [62]:
import pickle
output = open('members.pkl', 'wb')
pickle.dump(members_df_all, output)

output.close()

### Swing state


In [ ]:
import requests

headers = {'X-API-Key': '######'}

def get_data(url):
    responses = []
    response = requests.get(url, headers = headers)
    #print(response.json())
    responses.append(response.json())
    try:
        num_results = response.json()['num_results']
        print(num_results)
        for req in range(0,num_results,20):
            response = requests.get(url + '{num}'.format(num = req), headers = headers)
            responses.append(response.json())
        return responses
    except(KeyError):
        return responses

In [ ]:
# Get party membership counts for all states (current Congress only)
from pandas.io.json import json_normalize
party_counts_by_state = 'https://api.propublica.org/congress/v1/states/members/party.json'
state_party = get_data(party_counts_by_state)

In [ ]:
import pandas as pd
chambers = pd.DataFrame()
for cham in ['house', 'senate']:
    keys, values = [], []
    for state in state_party[0]['results'][cham]:
        for key, value in state.items():
            keys.append(key)
            values.append(value[0])
    chamber = pd.DataFrame(values, index = keys)
    chamber = chamber.add_prefix('{cham}_'.format(cham = cham))
    if chambers.empty is True:
        chambers = chamber
    else:
        chambers = pd.concat([chambers, chamber], axis = 1, join = 'outer')
chambers = chambers.fillna(0)

FiveThirtyEight identifies the states of Colorado, Florida, Iowa, Michigan, Minnesota, Ohio, Nevada, New Hampshire, North Carolina, Pennsylvania, Virginia, and Wisconsin as "traditional" swing states that have regularly seen close contests over the last few presidential campaigns.

https://fivethirtyeight.com/features/the-odds-of-an-electoral-college-popular-vote-split-are-increasing/


In [ ]:
chambers['swing_state'] = (chambers.index.isin(
    ['CO', 'FL', 'IA', 'MI', 'MN', 'OH', 'NV', 'NH', 'NC', 'PA', 'VI', 'WI']))

In [ ]:
# red states / blue states
import requests
election_results = 'https://en.wikipedia.org/wiki/List_of_United_States_presidential_election_results_by_state'
response = requests.get(election_results)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, 'html')

In [ ]:
list_states = (['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
                'Colorado', 'Connecticut', 'Delaware',' D.C.', 'Florida', 
                ' Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 
                'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
                'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 
                'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
                'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
                'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 
                'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
                'West Virginia', 'Wisconsin', 'Wyoming'])

In [ ]:
# red state if voted Rep >= 3 times in the last 4 elections
# blue state if voted Dem >= 3 times in the last 4 elections
# swing state if voted twice for both
from collections import Counter

red_states, blue_states, swing_states = [], [], []
for state in list_states:
    tds = [x.parent.find_next_siblings('td') for x in soup.find_all("a", text = state)]
    most_recent_4_elections = [x.text.strip('\n') for x in tds[0]][-4:]
    c = Counter(most_recent_4_elections)
    if c['R'] >= 3:
        red_states.append(state)
    elif c['D'] >= 3:
        blue_states.append(state)
    else:
        swing_states.append(state)

In [ ]:
us_state_abbrev = ({'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 
                   'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
                   'Connecticut': 'CT', 'Delaware': 'DE', ' D.C.': 'DC', 'Florida': 'FL', 
                   ' Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 
                   'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 
                   'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 
                   'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
                   'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
                   'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 
                   'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 
                   'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
                   'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 
                   'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 
                   'South Carolina': 'SC', 'South Dakota': 'SD', 
                   'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 
                   'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 
                   'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'})

In [ ]:
colors_states_abbr = []
for color in [red_states, blue_states, swing_states]:
    states_abbr = []
    for state in color:
        states_abbr.append(us_state_abbrev[state])
    colors_states_abbr.append(states_abbr)

In [ ]:
chambers['red_state'] = chambers.index.isin(colors_states_abbr[0])
chambers['blue_state'] = chambers.index.isin(colors_states_abbr[1])
chambers['swing_state2'] = chambers.index.isin(colors_states_abbr[2])

In [ ]:
chambers.reset_index(inplace = True)

In [ ]:
chambers.rename(columns = {'index': 'state'}, inplace = True)

In [ ]:
chambers = chambers[['state', 'swing_state', 'red_state', 'blue_state', 'swing_state2']]

In [ ]:
import pickle
output = open('states.pkl', 'wb')
pickle.dump(chambers, output)

output.close()

### Merging Data

In [90]:
# Merge members and swing states
import pickle

pkl_file = open('members.pkl', 'rb')
members = pickle.load(pkl_file)

pkl_file = open('states.pkl', 'rb')
states = pickle.load(pkl_file)

members_states = members.merge(states, left_on = 'members_state', right_on = 'state', how = 'left')

# Democrat in red state or Republican in blue state
members_states['member_diff_party_state'] = 0
members_states.loc[(members_states['red_state'] == True) & (members_states['members_party'] == 'D'),'member_diff_party_state'] = 1
members_states.loc[(members_states['blue_state'] == True) & (members_states['members_party'] == 'R'),'member_diff_party_state'] = 1

members_states.drop(columns = ['state', 'red_state', 'blue_state'], inplace = True)

In [65]:
# Merge mentioned bills, votes and topics (=bills_votes_topics)
import pickle

pkl_file = open('mentioned_bills.pkl', 'rb')
mentioned_bills = pickle.load(pkl_file)

pkl_file = open('votes_by_party.pkl', 'rb')
votes = pickle.load(pkl_file)
votes['bill_id'] = [vote.split('-')[0] for vote in votes['bill_id']]

bills_votes = mentioned_bills.merge(votes, left_on = 'votes_api_url', right_on = 'vote_api', how = 'inner')

In [76]:
bills_votes['congress'] = bills_votes['congress'].map(float)

In [77]:
bills_votes['congress'].value_counts()

114.0    320175
115.0    245738
113.0    212937
Name: congress, dtype: int64

In [78]:
# Merge speeches_metadata and bills_votes
pkl_file = open('speeches_metadata_evidence.pkl', 'rb')
speeches_metadata_evidence = pickle.load(pkl_file)

speeches_all_bill_data = speeches_metadata_evidence.merge(bills_votes, left_on = ['bill_id', 'congress'], right_on = ['bill_slug', 'congress'], how = 'inner')

In [81]:
import pickle
output = open('speeches_all_bill_data.pkl', 'wb')
pickle.dump(speeches_all_bill_data, output, protocol=4)

output.close()

In [92]:
members_states['members_congress'] = members_states['members_congress'].map(float)

In [93]:
# Merge speeches_metadata and members_states
# Voters
speeches_bills_members = speeches_all_bill_data.merge(members_states, left_on = ['member_id', 'congress'], right_on = ['members_member_id', 'members_congress'], how = 'left', suffixes = ('_members','_voters'))
# Speaker
speaker_columns = ['members_chamber', 'members_senate_class',
       'members_seniority', 'members_state_rank', 'members_leadership_role',
       'members_state', 'members_votes_with_party_pct', 'members_party',
       'members_member_id', 'members_gender', 'members_congress']
#speeches_bills_members = speeches_bills_members.merge(members_states[speaker_columns], left_on = ['speaker_bioguide'], right_on = ['members_member_id'], how = 'left', suffixes = ('_voters','_speaker'))
# Sponsor
members_states = members_states[speaker_columns].add_suffix('_sponsor')
speeches_bills_members = speeches_bills_members.merge(members_states, left_on = ['sponsor_id', 'congress'], right_on = ['members_member_id_sponsor', 'members_congress_sponsor'], how = 'left')

In [ ]:
# Number of mentions of same / opposite affiliation think tank
party_abbr_dict = {'Republican': 'R', 'Democrat': 'D'}
speeches_bills_members['opp_affiliation_tt_mentioned_no_mean'] = 0
speeches_bills_members['same_affiliation_tt_mentioned_no_mean'] = 0
for key, value in party_abbr_dict.items():
    for idx in range(0,speeches_bills_members.shape[0]):
        if speeches_bills_members.loc[idx,'members_party'] == value:
            print(speeches_bills_members.loc[idx,'members_party'])
            print(speeches_bills_members.loc[idx,'mentions_think_tank_{party}_no_mean'.format(party = key)])
            speeches_bills_members.at[idx,'same_affiliation_tt_mentioned_no_mean'] = speeches_bills_members.loc[idx,'mentions_think_tank_{party}_no_mean'.format(party = key)]
        elif (speeches_bills_members.loc[idx,'mentions_think_tank_{party}_no_mean'.format(party = key)] !=0) and (speeches_bills_members.loc[idx,'members_party'] != value) and (speeches_bills_members.loc[idx,'members_party'] != 'I'):
            speeches_bills_members.at[idx,'opp_affiliation_tt_mentioned_no_mean'] = speeches_bills_members.loc[idx,'mentions_think_tank_{party}_no_mean'.format(party = key)]        

In [94]:
import pickle
output = open('speeches_bills_members.pkl', 'wb')
pickle.dump(speeches_bills_members, output)

output.close()